In [32]:
import pandas as pd
import numpy
import scanpy as sc
from sklearn.preprocessing import StandardScaler
import torch
import pickle

In [2]:
adata = sc.read('Data/dataset_annotated.h5ad')

In [3]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000) 

/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:305: RuntimeWarning: invalid value encountered in log
  dispersion = np.log(dispersion)


In [4]:
adata_subset = adata[:, adata.var['highly_variable']].copy()

In [5]:
adata_subset

AnnData object with n_obs × n_vars = 3273 × 5000
    obs: 'scan-b external id', 'instrument model', 'age at diagnosis', 'tumor size', 'lymph node group', 'lymph node status', 'er status', 'pgr status', 'her2 status', 'ki67 status', 'nhg', 'er prediction mgc', 'pgr prediction mgc', 'her2 prediction mgc', 'ki67 prediction mgc', 'nhg prediction mgc', 'er prediction sgc', 'pgr prediction sgc', 'her2 prediction sgc', 'ki67 prediction sgc', 'pam50 subtype', 'overall survival days', 'overall survival event', 'endocrine treated', 'chemo treated', 'leiden', 'cluster_names', 'grade', 'classification'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'age at diagnosis_colors', 'classification_colors', 'cluster_names_colors', 'er status_colors', 'grade_colors', 'her2 status_colors', 'hvg', 'instrument model_colors', 'ki67 status_colors', 'leiden', 'lymph node group_colors', 'lymph node status_colors', 'neighb

In [15]:
df=sc.get.obs_df(adata_subset, keys=adata_subset.var_names.tolist())

In [16]:
ss=StandardScaler()

In [18]:
train_data = ss.fit_transform(df)

In [23]:
train_df=pd.DataFrame(train_data, index=df.index, columns=df.columns)

In [28]:
# 3. Train/Validation Split
print("Splitting data into train and validation sets...")
X_train, X_val = train_test_split(train_data, test_size=0.15, random_state=42) # 15% for validation
input_dim = X_train.shape[1]
n_samples_train = X_train.shape[0]
n_samples_val = X_val.shape[0]

print(f"Input dimension (number of genes): {input_dim}")
print(f"Training samples: {n_samples_train}, Validation samples: {n_samples_val}")

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
X_val_tensor = torch.FloatTensor(X_val)


Splitting data into train and validation sets...
Input dimension (number of genes): 5000
Training samples: 2782, Validation samples: 491


In [30]:
files={
    'full_dataset':train_df,
    'X_train':X_train,
    'X_val':X_val,
    'input_dim':input_dim,
    'n_samples_train':n_samples_train,
    'n_samples_val':n_samples_val,
    'X_train_tensor':X_train_tensor,
    'X_val_tensor':X_val_tensor
    
}

In [33]:
with open('Data/training_data.pkl', "wb") as f:
    pickle.dump(files, f)